In [ ]:
#### import packages and housekeep ####

In [ ]:
import pandas as pd
import numpy as np
import datetime
import statsmodels.api as sm
import os
import sys
import logging
from Matcher import Matcher
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, KFold
import matplotlib.pyplot as plt
import seaborn as sns
import instaquery as iq
from piper.db.snowflake import SnowflakeDb
from datetime import date
from datetime import timedelta
import jinja2

In [ ]:
#### generate attributes ####

In [ ]:
def date_ref_table_gen (start_date, cohort_duration_days, num_cohorts, date_ref_df = pd.DataFrame(), date_ref_temp_df = pd.DataFrame()): #start_date_inclusive 
    for n in range(num_cohorts):
#         print(n)
        
        start_date_temp = (date.fromisoformat(start_date) + timedelta(days=(n*cohort_duration_days))).isoformat()
        visit_start_temp = (date.fromisoformat(start_date_temp) - timedelta(days=(cohort_duration_days))).isoformat()
        end_date_temp = (date.fromisoformat(start_date_temp) + timedelta(days=(cohort_duration_days-1))).isoformat()

#         print(start_date_temp)
        date_ref_temp_df.loc[n, 'visit_start'] = visit_start_temp
        date_ref_temp_df.loc[n, 'start_date'] = start_date_temp 
        date_ref_temp_df.loc[n,'end_date'] = end_date_temp
#         date_ref_temp_df = pd.DataFrame({"visit_start":visit_start_temp, "start_date":start_date_temp,"end_date":end_date_temp})
              
#         print(date_ref_temp_df)
        
        if date_ref_df.empty == True:
            date_ref_df = date_ref_temp_df 
        else:
            date_ref_df = pd.concat([date_ref_df, date_ref_temp_df]) 
            
        date_ref_temp_df = pd.DataFrame() #date_ref_temp_df.iloc[0:0]
        
    return date_ref_df
        

#         print(start_date_temp)
#         print(end_date_temp)
    

In [ ]:
date_ref_df = date_ref_table_gen ('2022-06-26', 28, 14) #2021-10-13

In [ ]:
date_ref_df

In [ ]:
iq.upload(date_ref_df, 'sandbox_db.dianedou.ott_incrementality_vprod_date_ref_df')

In [ ]:
params = {}
# params['user_base_table'] = user_base_table
# params['reporting_date'] = str(reporting_date)

In [ ]:
def df_user_base_query(start_date_temp, end_date_temp, visit_start_temp):
    base_table_query_temp = f"""set start_date = '{start_date_temp}'; set end_date = '{end_date_temp}'; set visit_start = '{visit_start_temp}'; """
    SnowflakeDb.singleton().execute_sql(base_table_query_temp)

    with open('OTT_feature_gen.sql', 'r') as f:
        sql = f.read()
        query_feature = jinja2.Template(sql).render(params=params)
        SnowflakeDb.singleton().execute_sql(query_feature)
    return 

In [ ]:
for row in range(0, date_ref_df.shape[0]): 
    print(row)
    cohort_temp = date_ref_df.iloc[row, ]
    print(cohort_temp)
    
    start_date_temp = cohort_temp['start_date']
    end_date_temp  = cohort_temp['end_date']
    visit_start_temp = cohort_temp['visit_start']
    
    df_user_base_query(start_date_temp, end_date_temp, visit_start_temp)
    try:
        previous_df = SnowflakeDb.singleton().to_dataframe("select * from sandbox_db.dianedou.OTT_matching_attributes_vprod")
        previous_df.columns = previous_df.columns.str.lower()
        previous_df.cohort = pd.to_datetime(previous_df.cohort)
        if pd.Timestamp(start_date_temp) not in list(set(previous_df.cohort.to_list())):
            SnowflakeDb.singleton().execute_sql('CREATE OR REPLACE table sandbox_db.dianedou.OTT_matching_attributes_vprod as (select * from sandbox_db.dianedou.OTT_matching_attributes_vprod union all select * from sandbox_db.dianedou.OTT_matching_attributes_vprod_temp)')
    except:
        SnowflakeDb.singleton().execute_sql('CREATE OR REPLACE table sandbox_db.dianedou.OTT_matching_attributes_vprod as (select * from sandbox_db.dianedou.OTT_matching_attributes_vprod_temp)')


In [ ]:
#### generate outcome ####

In [ ]:
def df_outcome_query(cohort_temp):
    base_table_query_temp = f"""set cohort_temp = '{cohort_temp}'; set user_base_table = 'sandbox_db.dianedou.OTT_matching_attributes_vprod';"""
    
    SnowflakeDb.singleton().execute_sql(base_table_query_temp)

    with open('OTT_outcome_gen.sql', 'r') as f:
        sql = f.read()
        query_feature = jinja2.Template(sql).render(params=params)
        SnowflakeDb.singleton().execute_sql(query_feature)
    return 

In [ ]:
for row in range(1, date_ref_df.shape[0]): 

    cohort_temp = date_ref_df.loc[row, 'start_date']
    print(cohort_temp)

    df_outcome_query(cohort_temp)
    
    try:
        previous_df = SnowflakeDb.singleton().to_dataframe("select * from sandbox_db.dianedou.OTT_reporting_metrics_28d_vprod_v2")
        previous_df.columns = previous_df.columns.str.lower()
        previous_df.cohort = pd.to_datetime(previous_df.cohort)
        if pd.Timestamp(cohort_temp) not in list(set(previous_df.cohort.to_list())):
            SnowflakeDb.singleton().execute_sql('CREATE OR REPLACE table sandbox_db.dianedou.OTT_reporting_metrics_28d_vprod_v2 as (select * from sandbox_db.dianedou.OTT_reporting_metrics_28d_vprod_v2 union all select * from sandbox_db.dianedou.OTT_reporting_metrics_28d_vprod_temp)')
            SnowflakeDb.singleton().execute_sql('CREATE OR REPLACE table sandbox_db.dianedou.OTT_reporting_metrics_orders_28d_vprod_v2 as (select * from sandbox_db.dianedou.OTT_reporting_metrics_orders_28d_vprod_v2 union all select * from sandbox_db.dianedou.OTT_reporting_metrics_orders_28d_vprod_temp)')
    except:
        SnowflakeDb.singleton().execute_sql('CREATE OR REPLACE table sandbox_db.dianedou.OTT_reporting_metrics_28d_vprod_v2 as (select * from sandbox_db.dianedou.OTT_reporting_metrics_28d_vprod_temp)')
        SnowflakeDb.singleton().execute_sql('CREATE OR REPLACE table sandbox_db.dianedou.OTT_reporting_metrics_orders_28d_vprod_v2 as (select * from sandbox_db.dianedou.OTT_reporting_metrics_orders_28d_vprod_temp)')


---------------------

In [ ]:
#### matching ####

In [ ]:
df = iq.query("SELECT * FROM sandbox_db.dianedou.ott_matching_attributes_vprod")

In [ ]:
feature = ['total_spend_fav_store_last_five_deliveries'
            , 'tot_visit_time'
            , 'n_platforms_visited_l28'
            , 'days_signup_to_activation'
            , 'min_days_between_reorder_l28'
            , 'avg_initial_charge_amt_usd_fav_store_last_five_deliveries'
            , 'total_items_fav_store_last_five_deliveries'
            , 'days_since_last_order_fav_store_last_five_deliveries'
            , 'min_days_between_reorder'
            , 'days_since_last_completed_order'
            , 'avg_change_tip_pct'
            , 'past_trial'
            , 'tip'
            , 'is_wao'
            , 'gtv_l28'
            , 'gtv_l91'
            , 'gtv_lifetime'
            , 'deliveries_l28'
            , 'deliveries_lifetime'
            , 'is_hao'
            , 'visits_l28'
            , 'visits_l91'
            , 'visits_lifetime'
            , 'signup_days'
            , 'zip_median_income'
          ]

In [ ]:
df_intervention = df.copy()
obj_df = df_intervention.select_dtypes(include=['object']).copy()

In [ ]:
def remove_elements_from_list (list_keep, list_remove):
    for i in list_remove:
        try:
            list_keep.remove(i)
        except:
            print(i + ' does not exist')
    return list_keep

In [ ]:
def rename_cols(df):
    df.columns = np.char.replace(np.char.replace(np.char.lower(df.columns.values.astype('str')), ':overall::numeric(10,2)',''), 'fuga.','')


In [ ]:
dummy_var = remove_elements_from_list (obj_df.columns.to_list() , [ 'wk_ref', 'created_at_pt', 'cohort', 'measured_date','activation_warehouse_name'])

In [ ]:
X_coded = pd.get_dummies(df_intervention, columns = dummy_var, drop_first = True)
# df_intervention_lite = pd.concat([df_intervention[reporting_metric + variant], X_coded], axis = 1)

In [ ]:
rename_cols(X_coded)

In [ ]:
X_coded = X_coded[feature]

In [ ]:
df_intervention_matching = pd.concat([df_intervention[['user_id', 'variant', 'cohort', 'wk_ref']], X_coded], axis = 1)

In [ ]:
df_intervention_matching.variant

In [ ]:
attributes_to_exclude = remove_elements_from_list (df_intervention_matching.columns.tolist() , feature)

In [ ]:
def matching_cohort (data, variant_var = 'variant'):

    control = data[data[variant_var] == 0].reset_index(drop=True)
    test = data[data[variant_var] == 1].reset_index(drop=True)

    m = Matcher(test, control, yvar=variant_var, exclude=attributes_to_exclude)
    np.random.seed(20170925)
    m.fit_scores(balance=True, nmodels=10)
    m.predict_scores()
    m.match(method="random", nmatches=1, threshold=0.0005)
    matched_df = m.matched_data.reset_index(drop=True)

    return (matched_df)


In [ ]:
cohort_list = list(set(df['cohort'].tolist()))

In [ ]:
sorted(cohort_list)

In [ ]:
for i in range(0, len(cohort_list)): #
    cohort_temp = sorted(cohort_list)[i]
    print(cohort_temp)
    df_intervention_matching_cohort = df_intervention_matching[df_intervention_matching['cohort'] == cohort_temp]
    df_matched = matching_cohort(df_intervention_matching_cohort)
    iq.upload(df_matched, 'sandbox_db.dianedou.OTT_matching_attributes_vprod_matched_temp')
    
    SnowflakeDb.singleton().execute_sql('CREATE OR REPLACE table sandbox_db.dianedou.OTT_matching_attributes_vprod_matched_sup as (select * from sandbox_db.dianedou.OTT_matching_attributes_vprod_matched_sup union all select * from sandbox_db.dianedou.OTT_matching_attributes_vprod_matched_temp)')
         